In [ ]:
!pip install transformers lightning pytorch-lightning lightning-bolts -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.0/330.0 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Generating data to overfit LongT5

In [ ]:
import pandas as pd
df = pd.read_csv("longt5_dataset_1.tsv", header=None, sep="\t")

In [ ]:
import random

def get_sample_pair(prompt, target, n_samples):
  shuffled_samples = []
  splitted_target = target.split(">")
  for i in range(n_samples):
    shuffled_samples.append([int(splitted_target[i].strip()[1]), i])
  splitted_prompt = prompt.split("\n")

  new_prompt = splitted_prompt[:2]
  random.shuffle(shuffled_samples)

  new_idxs_ranking = [0] * n_samples
  for idx, sample in enumerate(shuffled_samples):
    doc, rank = sample
    splitted_prompt[doc + 1] = f"[doc_{idx + 1}] {splitted_prompt[doc + 1][3:]}"
    new_prompt.append(splitted_prompt[doc + 1])
    new_idxs_ranking[rank] = f"doc_{idx + 1}"
  new_prompt.extend(splitted_prompt[-3:])
  target = ""
  for rank in new_idxs_ranking:
    target += f"[{rank}] > "
  output = ["\n".join(new_prompt), target[:-3]]

  output[0] = output[0].replace("10 passages", f"{n_samples} passages")
  return output

In [ ]:
overfit_data = []
for i in range(2, 10):
  text_prompts = []
  targets = []
  for idx, row in df.iterrows():
    try:
      text_prompt, target = get_sample_pair(row[0], row[1], i)
      text_prompts.append(text_prompt)
      targets.append(target)
    except:
      continue
  overfit_data.append([text_prompts, targets, i])

In [ ]:
for text_prompts, targets, n_samples in overfit_data:
  df_tmp = pd.DataFrame()
  df_tmp["prompt"] = text_prompts
  df_tmp["target"] = targets
  df_tmp.to_csv(f"longt5_data_{n_samples}_samples.tsv", sep="\t", index=False)

## Training with 9 documents

In [ ]:
!python train.py \
 --pretrained_model_name_or_path google/long-t5-tglobal-base \
 --data_path /content/longt5_data_9_samples.tsv

2023-06-28 15:33:48.400279: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /content/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                           | Params
-----------------

## Pushing to HF hub

In [ ]:
from huggingface_hub import login
from train import ListwiseT5

login()

In [ ]:
model = ListwiseT5.load_from_checkpoint("/content/checkpoints/model-epoch=25-val_loss=0.00.ckpt")

In [ ]:
model.model.push_to_hub("pedrogengo/listwise_longt5_1k_msmarco")

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/pedrogengo/listwise_longt5_1k_mmarco/commit/417faf53e302a6632507d66fbcd42f8f10b284e8', commit_message='Upload LongT5ForConditionalGeneration', commit_description='', oid='417faf53e302a6632507d66fbcd42f8f10b284e8', pr_url=None, pr_revision=None, pr_num=None)